In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from itertools import chain
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
#import keras_tuner as kt
df = pd.read_csv("drive/MyDrive/Engineer's Project/test.csv")

In [ ]:
df.pop('usa_cpi')
df.pop('pol_cpi')
df.pop('usa_inter')
df.pop('pol_inter')
df.pop('Date')

0       2010.11.15
1       2010.11.16
2       2010.11.17
3       2010.11.18
4       2010.11.19
           ...    
3537    2022.03.27
3538    2022.03.28
3539    2022.03.29
3540    2022.03.30
3541    2022.03.31
Name: Date, Length: 3542, dtype: object

In [ ]:
scaler = MinMaxScaler()
df_numpy = df.to_numpy() 
scaler = scaler.fit(df_numpy[2000:3000])
df_scalled = scaler.transform(df_numpy)
df_scalled = pd.DataFrame(df_scalled, columns=[
  'Opening', 'High', 'Low', 'Closing','Momentum', 'Range', 'ohlc'])

In [ ]:
df_train = df_scalled[2200:3001]
df_val = df_scalled[2000:2200] #300
df_game = df_scalled[3001:]

In [ ]:
lookback = 15
step = 1
delay = 0
batch_size = 128

In [ ]:
len(df_train)

801

In [ ]:
len(df_val)

200

In [ ]:
len(df_game)

In [ ]:
float_data = np.array(df_scalled).astype('float32')

In [ ]:
float_train_data = np.array(df_train).astype('float32')

In [ ]:
float_val_data = np.array(df_val).astype('float32')

In [ ]:
float_game_data = np.array(df_game).astype('float32')

In [ ]:
len(float_val_data)

200

In [ ]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(
        min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][3] 
    yield samples, targets

In [ ]:
train_gen = generator(float_train_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=800,
#shuffle=True,
step=step,
batch_size=batch_size)

In [ ]:
val_gen = generator(float_val_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=199,
step=step,
batch_size=batch_size)

In [ ]:
test_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=3001,
max_index=3541,
step=step,
batch_size=batch_size)

In [ ]:
train_steps = (801 - lookback)
val_steps = (200 - lookback)
##test_steps = (3543 - 3001 - lookback)

In [ ]:
del model

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.LSTM(120,
  activation='sigmoid',
  dropout=0.1,
  recurrent_dropout=0.1,
  input_shape=(None, float_train_data.shape[-1])))
model.add(layers.Dense(160,
  activation='sigmoid',))
model.add(layers.Dense(1,
  activation='sigmoid'))
model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
history = model.fit(train_gen,
  steps_per_epoch=200,
  epochs=50,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/200
200/200 [==============================] - 15s 62ms/step - loss: 0.1448 - val_loss: 0.1273
Epoch 2/200
200/200 [==============================] - 12s 62ms/step - loss: 0.0738 - val_loss: 0.0478
Epoch 3/200
200/200 [==============================] - 15s 74ms/step - loss: 0.0641 - val_loss: 0.0775
Epoch 4/200
200/200 [==============================] - 14s 72ms/step - loss: 0.0555 - val_loss: 0.0792
Epoch 5/200
200/200 [==============================] - 15s 78ms/step - loss: 0.0481 - val_loss: 0.0224
Epoch 6/200
200/200 [==============================] - 13s 63ms/step - loss: 0.0426 - val_loss: 0.0446
Epoch 7/200
200/200 [==============================] - 15s 76ms/step - loss: 0.0414 - val_loss: 0.0192
Epoch 8/200
200/200 [==============================] - 13s 67ms/step - loss: 0.0398 - val_loss: 0.0396
Epoch 9/200
200/200 [==============================] - 15s 76ms/step - loss: 0.0387 - val_loss: 0.0478
Epoch 10/200
200/200 [==============================] - 13s 66ms/step - l

In [ ]:
del model

Calculate correct peaks and value that is not scaled back.

In [ ]:
model = tf.keras.models.load_model("drive/MyDrive/Engineer's Project/best_with_GRU2.h5")

In [ ]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, None, 120)         61440     
                                                                 
 gru_17 (GRU)                (None, 5)                 1905      
                                                                 
 dense_27 (Dense)            (None, 160)               960       
                                                                 
 dense_28 (Dense)            (None, 1)                 161       
                                                                 
Total params: 64,466
Trainable params: 64,466
Non-trainable params: 0
_________________________________________________________________


In [ ]:
len(float_game_data)

0

In [ ]:
correct_peaks1 = 0
value1 = 0
for i in range(0, 541):
  last151 = float_train_data[-lookback:]
  last151 = last151.reshape(1,15,7)
  prediction1 = model.predict(last151)
  last_val1 = last151[0][lookback-1][3]
  actual_val1 = float_game_data[0][3]
  if((prediction1 > last_val1) and (actual_val1 > last_val1)):
    correct_peaks1+=1
    value1 +=  100 *(actual_val1 - last_val1)
  elif((prediction1 < last_val1) and (actual_val1 < last_val1)):
    correct_peaks1+=1
    value1 += 100 * (last_val1 - actual_val1)
  else:
    value1 -= 100 * abs(actual_val1 - last_val1)
  float_train_data = np.vstack((float_train_data, float_game_data[0]))
  float_game_data = np.delete(float_game_data, 0, 0)


In [ ]:
correct_peaks1

274

In [ ]:
value1 * scaler.scale_[3]

267.337342927077

In [ ]:
scaler.scale_ * value1

array([2.62655469e+03, 2.56125347e+03, 2.68934384e+03, 2.62644947e+03,
       9.49858034e+06, 1.56653898e+04, 2.64679502e+03])

In [ ]:
scaler.scale_

array([1.05629744e+00, 1.03003585e+00, 1.08154878e+00, 1.05625512e+00,
       3.81995704e+03, 6.30000630e+00, 1.06443731e+00])

In [ ]:
def return_model():
  model = tf.keras.models.Sequential()
  model.add(layers.LSTM(120,
    activation='sigmoid',
    #dropout=0.1,
    #recurrent_dropout=0.1,
    return_sequences=True,
    input_shape=(None, float_train_data.shape[-1])))
  model.add(layers.GRU(5,
    activation='sigmoid',
    input_shape=(None, float_train_data.shape[-1])))
  model.add(layers.Dense(160,
    activation='sigmoid',))
  model.add(layers.Dense(1,
    activation='sigmoid'))
  model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
  return model

version 2 creates model after lookback iterations

In [ ]:
del model

In [ ]:
correct_peaks = 0
value = 0
for i in range(0, 541):##441
  last15 = float_train_data[-lookback:]
  last15 = last15.reshape(1,15,7)
  prediction = model.predict(last15)
  last_val = last15[0][lookback-1][3]
  actual_val = float_game_data[0][3]
  if((prediction > last_val) and (actual_val > last_val)):
    correct_peaks+=1
    value += 100 * (actual_val - last_val)
  elif((prediction < last_val) and (actual_val < last_val)):
    correct_peaks+=1
    value += 100 * (last_val - actual_val)
  else:
    value -= 100 * abs(actual_val - last_val)
  float_train_data = np.vstack((float_train_data, float_game_data[0]))
  float_game_data = np.delete(float_game_data, 0, 0)

  if i == 200:
    del model
    del train_gen
    model = return_model()
    train_gen = generator(float_train_data,
      lookback=lookback,
      delay=delay,
      min_index=0,
      max_index=len(float_train_data) - 1,
      #shuffle=True,
      step=step,
      batch_size=batch_size)
    model.fit(train_gen,
      steps_per_epoch=200,
      epochs=200,
      validation_data=val_gen,
      validation_steps=val_steps)
  


Epoch 1/200
200/200 [==============================] - 13s 54ms/step - loss: 0.1703 - val_loss: 0.2248
Epoch 2/200
200/200 [==============================] - 11s 53ms/step - loss: 0.1493 - val_loss: 0.0752
Epoch 3/200
200/200 [==============================] - 11s 53ms/step - loss: 0.1103 - val_loss: 0.2105
Epoch 4/200
200/200 [==============================] - 10s 52ms/step - loss: 0.0968 - val_loss: 0.0644
Epoch 5/200
200/200 [==============================] - 11s 53ms/step - loss: 0.0882 - val_loss: 0.1925
Epoch 6/200
200/200 [==============================] - 11s 53ms/step - loss: 0.0814 - val_loss: 0.1310
Epoch 7/200
200/200 [==============================] - 12s 61ms/step - loss: 0.0722 - val_loss: 0.0712
Epoch 8/200
200/200 [==============================] - 11s 53ms/step - loss: 0.0644 - val_loss: 0.0735
Epoch 9/200
200/200 [==============================] - 11s 53ms/step - loss: 0.0611 - val_loss: 0.0576
Epoch 10/200
200/200 [==============================] - 12s 58ms/step - l

In [ ]:
correct_peaks

315

In [ ]:
value * scaler.scale_[3]

124.77533439076119

In [ ]:
skonczyl sie float_game_data - nie ma punktow

In [ ]:
correct_peaks = 0
value = 0
for i in range(0, 541):##441
  last15 = float_train_data[-lookback:]
  last15 = last15.reshape(1,15,7)
  prediction = model.predict(last15)
  last_val = last15[0][lookback-1][3]
  actual_val = float_game_data[0][3]
  if((prediction > last_val) and (actual_val > last_val)):
    correct_peaks+=1
    value += 100 * (actual_val - last_val)
  elif((prediction < last_val) and (actual_val < last_val)):
    correct_peaks+=1
    value += 100 * (last_val - actual_val)
  else:
    value -= 100 * abs(actual_val - last_val)
  float_train_data = np.vstack((float_train_data, float_game_data[0]))
  float_game_data = np.delete(float_game_data, 0, 0)
  del train_gen2
  train_gen2 = generator(float_train_data[-lookback-2:],
      lookback=lookback,
      delay=delay,
      min_index=0,
      max_index=len(float_train_data[-lookback-2:]) - 1,
      #shuffle=True,
      step=step,
      batch_size=batch_size)

  model.fit(train_gen2)


    743/Unknown - 12s 16ms/step - loss: 0.0032

KeyboardInterrupt: ignored

In [ ]:
train_genTest = generator(float_train_data[-lookback-1:],
lookback=lookback,
delay=delay,
min_index=0,
max_index=14,
#shuffle=True,
step=step,
batch_size=16)

TypeError: ignored

In [ ]:
train_gen3 = generator(float_train_data,
      lookback=lookback,
      delay=delay,
      min_index=0,
      max_index=len(float_train_data) - 1,
      #shuffle=True,
      step=step,
      batch_size=batch_size)

In [ ]:
np.asarray(np.asarray(actual_val))

array(0.67154217, dtype=float32)

In [ ]:
train_gen2 = generator(float_train_data[-lookback-2:],
      lookback=lookback,
      delay=delay,
      min_index=0,
      max_index=len(float_train_data[-lookback-2:]) - 1,
      #shuffle=True,
      step=step,
      batch_size=batch_size)

In [ ]:
next(train_gen2)[0][0]

IndexError: ignored